<a href="https://colab.research.google.com/github/entangelk/study_data_analytics/blob/main/docs/quests/EDAs/EDA_ShoppingMallDeliveryWithDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한글 폰트 설치 및 시각화

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 런타임메뉴에서 리스타트 세션 해줘야함

- 한글 폰트 설치가 끝났으면 리스타트 잊지말라구!

In [1]:
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd

In [4]:
# 유니코드 마이너스 사용 설정
plt.rcParams['axes.unicode_minus'] = False

In [5]:
import numpy as np

## 시작

In [6]:
get_data = pd.read_csv('ShoppingMallDeliveryWithDate.csv')
get_data

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077,1,PC쇼핑몰,20181227
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42,2,모바일웹,20181229
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271,1,모바일웹,20181230
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085,1,모바일웹,20181231
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106,1,모바일웹,20181231
...,...,...,...,...,...,...,...,...,...
26947,무통장입금,D-20180723-0000042-00,2018-07-24 오후 6:45,2018-07-28 오전 5:32,70500,42,1,PC쇼핑몰,20180723
26948,무통장입금,D-20180723-0000042-00,2018-07-24 오후 6:45,2018-07-28 오전 5:32,0,1209,1,PC쇼핑몰,20180723
26949,무통장입금,D-20180723-0000052-00,2018-07-24 오전 9:50,2018-07-28 오전 5:32,184760,1036,2,PC쇼핑몰,20180723
26950,무통장입금,D-20180723-0000052-00,2018-07-24 오전 9:50,2018-07-28 오전 5:32,0,1209,1,PC쇼핑몰,20180723


### 연속형 -> 범주형 만들기 (상품구매금액)

In [12]:
get_data.query('상품구매금액 != 0')['상품구매금액'].sort_values()

1843         700
23845       2370
20806       2500
1150        3590
13391       3670
          ...   
1541     1325000
8744     1470750
2615     1691250
20642    1920000
24512    4963400
Name: 상품구매금액, Length: 24062, dtype: int64

In [17]:
bins = [-1,0,9999,49999,99999,499999,9999999999]
labels = ['0','1-10000','10001-50000','50001-100000','100001-500000','500000+']
get_data['상품구매금액_category'] = pd.cut(x=get_data['상품구매금액'], bins=bins, labels=labels)

In [18]:
get_data['상품구매금액_category']

0         50001-100000
1        100001-500000
2          10001-50000
3         50001-100000
4         50001-100000
             ...      
26947     50001-100000
26948                0
26949    100001-500000
26950                0
26951    100001-500000
Name: 상품구매금액_category, Length: 26952, dtype: category
Categories (6, object): ['0' < '1-10000' < '10001-50000' < '50001-100000' < '100001-500000' <
                         '500000+']

In [21]:
get_data['상품구매금액_category'].value_counts().sort_index()

0                 2890
1-10000            207
10001-50000       3399
50001-100000     14955
100001-500000     5478
500000+             23
Name: 상품구매금액_category, dtype: int64